In [46]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, DateType, StringType


spark = SparkSession.builder.master("local[1]")\
          .appName("task1")\
          .getOrCreate()


df_accounts = spark.read.option("header",True) \
     .option("delimiter", ";")\
     .csv("work/accounts.csv")

df_country = spark.read.option("header",True) \
    .option("delimiter", ";")\
     .csv("work/country_abbreviation.csv")

df_trans = spark.read.option("header",True) \
    .option("delimiter", ";")\
     .csv("work/transactions.csv")

In [2]:
df_trans.take(5)

[Row(id='179528', amount='-730.86', account_type='Business', transaction_date='2013-07-10', country='SV'),
 Row(id='378343', amount='-946.98', account_type='Personal', transaction_date='2018-04-06', country='YE'),
 Row(id='75450', amount='7816.92', account_type='Professional', transaction_date='2016-11-20', country='SI'),
 Row(id='357719', amount='704.02', account_type='Business', transaction_date='2016-11-06', country='ID'),
 Row(id='110511', amount='3462.6', account_type='Personal', transaction_date='2018-01-18', country='BS')]

In [5]:
df_trans.createOrReplaceTempView("trans")

In [6]:
df_country.createOrReplaceTempView("accounts")

In [23]:
df11 = spark.sql("SELECT account_type, COUNT(account_type) as account_type_count FROM \
    (SELECT id, account_type, COUNT(account_type) FROM trans group by id, account_type)\
    GROUP BY account_type")

In [24]:
df11.describe()

DataFrame[summary: string, account_type: string, account_type_count: string]

In [28]:
df11 = df11.withColumn("account_type_count", df11["account_type_count"].cast(IntegerType()))
df11.drop("summary")

DataFrame[account_type: string, account_type_count: int]

In [30]:
df11.show(3)

+------------+------------------+
|account_type|account_type_count|
+------------+------------------+
|    Personal|            481997|
|Professional|            482170|
|    Business|            482350|
+------------+------------------+



In [50]:
df12 = spark.sql("SELECT id as account_id, SUM(amount) as balance, MAX(transaction_date) as latest_date FROM trans group by id")

In [41]:
df12.describe()

DataFrame[summary: string, account_id: string, balance: string, latest_date: string]

In [42]:
df12.head(5)

[Row(account_id='1', balance=51909.75000000001, latest_date='2021-08-19'),
 Row(account_id='10', balance=50807.19, latest_date='2020-12-27'),
 Row(account_id='100', balance=56948.54, latest_date='2018-06-03'),
 Row(account_id='1000', balance=59101.64, latest_date='2020-06-24'),
 Row(account_id='10000', balance=69802.32, latest_date='2019-12-30')]

In [48]:
df12 = df12.withColumn("latest_date", df12["latest_date"].cast(DateType()))
df12 = df12.withColumn("balance", df12["balance"].cast(StringType()))

df12.drop("summary")

DataFrame[account_id: string, balance: string, latest_date: date]

In [49]:
df12.show()

+----------+------------------+-----------+
|account_id|           balance|latest_date|
+----------+------------------+-----------+
|         1| 51909.75000000001| 2021-08-19|
|        10|          50807.19| 2020-12-27|
|       100|          56948.54| 2018-06-03|
|      1000|          59101.64| 2020-06-24|
|     10000|          69802.32| 2019-12-30|
|    100000|          64058.75| 2020-11-11|
|    100001|           78541.7| 2021-12-06|
|    100002|           44956.1| 2021-07-21|
|    100003|          52879.78| 2021-09-09|
|    100004| 52040.31999999999| 2020-10-11|
|    100005|          42413.65| 2021-12-30|
|    100006|           40926.2| 2019-06-11|
|    100007|          56828.13| 2020-01-19|
|    100008|          35136.65| 2021-08-22|
|    100009|31456.100000000002| 2021-08-25|
|     10001|          39440.29| 2021-06-02|
|    100010|           35856.3| 2021-01-03|
|    100011|          94923.52| 2020-12-18|
|    100012|52974.729999999996| 2020-12-28|
|    100013|39774.490000000005| 